In [3]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Load your CSV
df = pd.read_csv("combined_good_coil_data_added gauge target and phases_Phase 2.csv")
# Preview first rows
df.head()

,Coil ID,Coil Length [30ms],Master Ramp,Shape Target Second Coefficient,Stand 1 Predicted Run Force,Stand 1 Gap Stick Offset,Tension Reel Calculated Tension,Stand 1-2 Total Tension Feedback,Stand 3 - Operator Side Force,Stand 1 Run Gap Setpoint,...,Stand 4 DS Roll Force,Roll Force Hydraulics Pressure Feedback,Stand 1 Roll Force,Stand 1 Roll Force limit (g67 delayed 200 ms + g80),Stand 1 Roll Force Increase Limit (based on predicted run force),Stand 4 OS Bending Shape Trim,Stand 4 DS Bending Shape Trim,Shape Target Second Coefficient.1,X4 Gauge Target,Phase
0,5399358,694.3333,100.0,10.0,1769.0,0.0,9.481006,43.576996,701.00903,44.30005,...,380.87000,2979.6667,1658.0,1842.7,176.0,-40.258440,36.386100,10.0,0.022644,Phase 2
1,5399358,698.5000,100.0,10.0,1769.0,0.0,9.453998,43.220997,704.50684,44.30005,...,370.25995,2979.0000,1657.8,1838.8,176.0,-40.217850,36.423515,10.0,0.014760,Phase 2
2,5399358,702.0000,100.0,10.0,1769.0,0.0,9.411001,43.255993,704.13700,44.30005,...,380.01000,2978.6667,1670.0,1834.9,176.0,-40.187570,36.479492,10.0,0.007812,Phase 2
3,5399358,705.0000,100.0,10.0,1769.0,0.0,9.369995,44.102997,709.04280,44.30005,...,379.38004,2978.0000,1678.4,1832.6,176.0,-40.136982,36.552420,10.0,0.014652,Phase 2
4,5399358,709.2500,100.0,10.0,1769.0,0.0,9.350006,43.545000,702.64820,44.30005,...,380.26996,2977.0000,1684.0,1840.4,176.0,-40.113930,36.631176,10.0,0.025668,Phase 2


In [5]:
# Specify only the IBA signal columns (edit this list)
iba_signals = ['Shape Target Second Coefficient', 'Stand 1 Predicted Run Force', 'Stand 1 Gap Stick Offset', 'Tension Reel Calculated Tension', 
                    'Stand 1-2 Total Tension Feedback', 'Stand 3 - Operator Side Force', 'Stand 1 Run Gap Setpoint', 'Stand 1 Gap Bite Offset', 
                    'S1 Operating Bending Trim', 'Stand 2-3 Tension Reference', 'Stand 3 Predicted Run Force', 'Neet Oil Concentration', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand1', 'Stand 4 Top Current Feedback', 'Morgoil DriveTop Bearing Outflow Temp Stand3', 
                    'Stand 2-3 Total Tension Feedback', 'Stand 2 Predicted Run Force', 'Stand 4 Gap Thread Offset', 'Stand 2 - Operator Side Force', 
                    'Stand 3 Run Gap Setpoint', 'Stand 2 Total Bending Feedback', 'Stand 2 Gap Bite Offset', 'Morgoil OperBottom Bearing Outflow Temp Stand3', 
                    'Exit Tension Reel Tension Reference', 'Stand 4 Thread Gap Setpoint', 'X4 Gauge Deviation', 'Stand 4 DS Total Bending Feedback',
                    'Stand 4 Gap Stick Offset', 'Stand 4 - Operator Side Force', 'Stand 2 Gap Eccentricity Trim', 'Stand 4 Gap Operator Offset', 
                    'Stand 3 Total Bending Feedback', 'Morgoil OperTop Bearing Outflow Temp Stand1', 'Stand 1 - Operator Side Force', 'X1 Gauge Deviation', 
                    'Stand 3 Drive Speed Feedback', 'Stand 2 Gap Thread Offset', 'Stand 4 Gap Bite Offset', 'Stand 2 Drive Speed Feedback', 
                    'Stand 3 Thread Gap Setpoint', 'Stand 1 Drive Speed Feedback', 'Stand 1-3 Solution System Pressure', 'Morgoil OperBottom Bearing Outflow Temp Stand4', 
                    'Stand 2 Thread Gap Setpoint', 'Stand 2 Top Current Feedback', 'Stand 1-3 Solution Temperature', 'Stand 4 - Drive Side Force', 'Stand 1 - Drive Side Force', 
                    'AGC GE Feedforward Hardness Number', 'Stand 1 Total Bending Feedback', 'Morgoil OperBottom Bearing Outflow Temp Stand1', 'X0 Gauge Deviation', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand4', 'Stand 4 Predicted Run Force', 'Stand 3 Bottom Current Feedback', 'Stand 4 Gap Eccentricity Trim', 
                    'Morgoil DriveBottom Bearing Outflow Temp Stand1', 'Stand 2 Gap Stick Offset', 'Stand 3-4 Tension Reference', 'Stand 4 Bottom Current Feedback', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand2', 'Stand 1 Bottom Current Feedback', 'S3 Operating Bending Trim', 'Morgoil DriveBottom Bearing Outflow Temp Stand4', 
                    'Stand 4 Drive Speed Feedback', 'Stand 3 Gap Stick Offset', 'Morgoil OperTop Bearing Outflow Temp Stand4', 'Stand 3 Gap Thread Offset', 
                    'Morgoil OperBottom Bearing Outflow Temp Stand2', 'Stand 3-4 Total Tension Feedback', 'Morgoil OperTop Bearing Outflow Temp Stand2', 
                    'Stand 2 Gap Operator Offset', 'Stand 2 Bottom Current Feedback', 'Stand 2 - Total Force', 'Stand 4 Solution System Pressure', 'Stand 3 Gap Eccentricity Trim', 
                    'Stand 3 Gap Bite Offset', 'Stand 2 - Drive Side Force', 'Stand 4 OS Total Bending Feedback', 'Morgoil DriveBottom Bearing Outflow Temp Stand2', 
                    'Stand 4 Run Gap Setpoint', 'Stand 1-2 Tension Reference', 'Stand 1 Gap Thread Offset', 'Stand 1 Gap Operator Offset', 'AGC Alex Dynamic Feedforward Hardness Number',
                    'Stand 3 - Drive Side Force', 'Stand 2 Run Gap Setpoint', 'Stand 1 - Total Force', 'Stand 1 Thread Gap Setpoint', 'Stand 1 Top Current Feedback', 
                    'S4 Operating Bending Trim', 'Stand 1 Gap Eccentricity Trim', 'Master Ramp.1', 'Stand 3 Top Current Feedback', 'Morgoil DriveBottom Bearing Outflow Temp Stand3', 
                    'S2 Operating Bending Trim', 'Morgoil OperTop Bearing Outflow Temp Stand3', 'Stand 3 Gap Operator Offset', 'Stand 1 Backup RPM', 'Stand 2 Backup RPM', 
                    'Stand 3 Backup RPM', 'Stand 4 Backup RPM', 'Stand 1 Top Motor RPM', 'Stand 1 Bottom Motor RPM', 'Stand 2 Top Motor RPM', 'Stand 2 Bottom Motor RPM', 
                    'Stand 3 Top Motor RPM', 'Stand 3 Bottom Motor RPM', 'Stand 4 Top Motor RPM', 'Stand 4 Bottom Motor RPM', 'Payoff Reel OS RPM', 'Payoff Reel DS RPM', 
                    'Exit Tension Reel RPM', 'Roll Force Hydraulic Tank Level Inches', 'Stand 1 OS Roll Force', 'Stand 2 OS Roll Force', 'Stand 3 OS Roll Force', 
                    'Stand 4 OS Roll Force', 'Stand 1 DS Roll Force', 'Stand 2 DS Roll Force', 'Stand 3 DS Roll Force', 'Stand 4 DS Roll Force', 'Roll Force Hydraulics Pressure Feedback',
                    'Stand 1 Roll Force', 'Stand 1 Roll Force limit (g67 delayed 200 ms + g80)', 'Stand 1 Roll Force Increase Limit (based on predicted run force)', 
                    'Stand 4 OS Bending Shape Trim', 'Stand 4 DS Bending Shape Trim', 'Shape Target Second Coefficient.1']

print("Available columns:", df.columns.tolist())
print("Using signals:", iba_signals)

Available columns: ['Coil ID', 'Coil Length s]', 'Master Ramp', 'Shape Target Second Coefficient', 'Stand 1 Predicted Run Force', 'Stand 1 Gap Stick Offset', 'Tension Reel Calculated Tension', 'Stand 1-2 Total Tension Feedback', 'Stand 3 - Operator Side Force', 'Stand 1 Run Gap Setpoint', 'Stand 1 Gap Bite Offset', 'S1 Operating Bending Trim', 'Stand 2-3 Tension Reference', 'Stand 3 Predicted Run Force', 'Neet Oil Concentration', 'Morgoil DriveTop Bearing Outflow Temp Stand1', 'Stand 4 Top Current Feedback', 'Morgoil DriveTop Bearing Outflow Temp Stand3', 'Stand 2-3 Total Tension Feedback', 'Stand 2 Predicted Run Force', 'Stand 4 Gap Thread Offset', 'Stand 2 - Operator Side Force', 'Stand 3 Run Gap Setpoint', 'Stand 2 Total Bending Feedback', 'Stand 2 Gap Bite Offset', 'Morgoil OperBottom Bearing Outflow Temp Stand3', 'Exit Tension Reel Tension Reference', 'Stand 4 Thread Gap Setpoint', 'X4 Gauge Deviation', 'Stand 4 DS Total Bending Feedback', 'Stand 4 Gap Stick Offset', 'Stand 4 - Op

In [6]:
def detect_anomalies_fixed(series, signal_name, coil_id, order=(1,1,1), min_points=50, max_points=None):
    series = series.dropna().reset_index(drop=True)
    
    if len(series) < min_points:
        return {"coil_id": coil_id, "signal": signal_name, "anomaly_percentage": None, "order": order}
    
    try:
        arima_model = ARIMA(series, order=order)
        fitted = arima_model.fit()
        forecast = fitted.predict(start=0, end=len(series)-1)
        residuals = series - forecast
        
        threshold = 3 * np.std(residuals)
        anomalies = residuals[np.abs(residuals) > threshold]
        
        anomaly_percentage = len(anomalies) / len(series) * 100
        
        return {"coil_id": coil_id, "signal": signal_name, "anomaly_percentage": anomaly_percentage, "order": order}
    
    except Exception as e:
        print(f"⚠ Skipping Coil {coil_id}, Signal {signal_name} due to error: {e}")
        return {"coil_id": coil_id, "signal": signal_name, "anomaly_percentage": None, "order": order}

In [ ]:
results = []
total_coils = df["Coil ID"].nunique()
coil_count = 0

for coil_id, coil_group in df.groupby("Coil ID"):
    coil_count += 1
    print(f"\nProcessing Coil {coil_id} ({coil_count}/{total_coils})...")
    
    for signal in iba_signals:
        if signal in coil_group.columns:
            # Simple percentile-based anomaly detection
            # anomaly_percentile=90 -> ~10% anomalies, 95 -> ~5% anomalies
            res = detect_anomalies_fixed(coil_group[signal], signal, coil_id, max_points=200) # anomaly_percentile=90
            results.append(res)

results_df = pd.DataFrame(results)
print("\nAll coils processed.")
# results_df


Processing Coil 5399358 (1/608)...

Processing Coil 5399575 (2/608)...

Processing Coil 5400215 (3/608)...


In [ ]:
results_df.to_csv("coil_anomaly_percentages_arima.csv", index=False)
print("Saved to coil_anomaly_percentages_arima.csv")